This workbook will perform sensitivity analysis following the "Elemental Effects" method outlined in:   
    
    Saltelli, A.: Global Sensitivity Analysis: The Primer. Chapter 3. [online] Available from: http://doi.wiley.com/10.1111/j.1751-5823.2008.00062_17.x, 2008.
    


In [2]:
#First, import the required packages
import pandas as pd
import numpy as np
from BCBlues import BCBlues
import pdb
import matplotlib.pyplot as plt
import random
import itertools
from joblib import Parallel, delayed

In [3]:
#Next, we will try to select the best "trajectories" r that will optimize the spread of the input parameters. 
#To do this, we first need to define the number of input parameters k, the steps we want to explore (p), and
#the number of trajectories we want to "scan" to choose our r best trajectories.
r = 4
p = 4
#Define the levels to be explored
levels = [1/3,2/3]
M = 100
#Input our parameters file. We will calculate k from this, thetam is the first parameter that varies
params = pd.read_excel('params_BC_sensitivity.xlsx',index_col = 0)
k = len(params['thetam':].index) #Parameters that don't vary are above thetam
#parameters that are used in the hydrology module, and so require the flows to be calculated
hydro_params = params['thetam':'Emax'].index
#Now we define the value of delta, our step size
d = p/(2*(p-1))
#Now we define a lower triangle matrix B that will be used to determine the different trajectories
B = np.ones((k+1,k),dtype=float)
B[np.triu_indices(n=k+1,m=k)] = 0


In [199]:
#Now, let's build one trajectory. This will be a transformed matrix B* following 
#B∗ = Jk+1?1x∗ +??/2? ? ?? 2B−Jk+1?k D∗ +Jk+1?k ? ?? P∗?
J1 = np.ones((k+1,1),dtype=float)
Jk = np.ones((k+1,k),dtype=float)
xstar = np.zeros((1,k))
Dstar = np.zeros((k))
for j in range(k):
    xstar[0,j] = random.choice(levels)
    Dstar[j] = random.choice([1.,-1.])
#Dstar = [1,-1]
#xstar
J1*xstar+(d/2)*np.array([(2*B-Jk)*Dstar+Jk])


array([[[1.33333333, 1.        , 0.33333333, ..., 1.        ,
         1.        , 0.33333333],
        [0.66666667, 1.        , 0.33333333, ..., 1.        ,
         1.        , 0.33333333],
        [0.66666667, 0.33333333, 0.33333333, ..., 1.        ,
         1.        , 0.33333333],
        ...,
        [0.66666667, 0.33333333, 1.        , ..., 0.33333333,
         1.        , 0.33333333],
        [0.66666667, 0.33333333, 1.        , ..., 0.33333333,
         0.33333333, 0.33333333],
        [0.66666667, 0.33333333, 1.        , ..., 0.33333333,
         0.33333333, 1.        ]]])

In [73]:
#xstar
k = 4
Pstar = np.identity(k)
np.random.shuffle(Pstar)
Pstar

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [4]:
#Now, we are going to define the M trajectories among which we will evaluate
J1 = np.ones((k+1,1),dtype=float)
Jk = np.ones((k+1,k),dtype=float)
Bstar = {}
#In this loop we define each of the M trajectories using the procedure laid out by Saltelli
for i in range(M):
    xstar = np.zeros((1,k))
    Dstar = np.zeros((k))
    Pstar = np.identity(k)
    np.random.shuffle(Pstar)
    for j in range(k):
        xstar[0,j] = random.choice(levels)
        Dstar[j] = random.choice([1.,-1.])
    #Dstar = [1,-1]
    #xstar
    Bstar[i] = np.dot(J1*xstar+(d/2)*np.array([(2*B-Jk)*Dstar+Jk]),Pstar)
    

In [5]:
#Next, we find what the maximum spread is for our system to choose the best trajectories
dist = {}
for i in range(M):
    B1 = Bstar[i]
    for ii in range(M): 
        B2 = Bstar[ii]
        #pdb.set_trace()
        if ((ii,i) in dist.keys()) | (i == ii):
            pass
        else:
            dist[i,ii] = np.sum((Bstar[i] - Bstar[ii])**2)
#Once we have our distances for each of them, we need to choose the r trajectories we will actually use
#best_traj = np.zeros(r)
len(dist)

4950

In [6]:
#Now that we have the the distances, we need to choose the best r trajectories that will maximize the overall distance
#This ends up being a combinations problem - we need to find the maximum value from a combination of r distances
#pdb.set_trace()
best_dist = 0
combo = []
for lists in list(itertools.combinations(range(M),r)):
    #pdb.set_trace()
    sumsq = 0
    for i in list(itertools.combinations(lists,2)):
        sumsq += dist[i]**2
    testdist = np.sqrt(sumsq)
    #pdb.set_trace()
    if testdist > best_dist:
        #pdb.set_trace()
        best_dist = testdist
        combo = np.unique(lists)      
combo

array([12, 13, 62, 69])

Below is the overall structure of the code, using a toy model (just summing the values) that isn't very informative. But - shows us what we can do, on one trajectory

In [7]:
traj = Bstar[17][0]
x = params['thetam':].val*traj[0,:]
y = np.zeros((k+1,1))
#pdb.set_trace()
#Set up output dataframe to store the elemental effects
EE = pd.DataFrame(index = params['thetam':].index)
EE.loc[:,'EE'] = 0
delta = d

#For each row of the trajectory, we will run the model.
for j in range(len(traj[:,0])):
    #pdb.set_trace()
    #First, change the input values
    x = params['thetam':].val*traj[j,:]
    #i is what input was changed          
    i = x.index[np.where(sign !=0)[0][0]]
    #Second, run the model and put the output(s) in the y array
    y[j] = x.sum()
    #Then, see what changed and in which direction and calculate the elemental effect. 
    #Could pull from loop, but this isn't really going to impact performance on long model runs
    if j == 0: #No change in first run
        pass
    else:#i is what input was changed  
        sign = np.sign((traj[j,:] - traj[j-1,:]))
        #Calculate the elemental effect - using normalized change in parameter will always be delta (d)
        EE.loc[i,'EE'] = sign.sum()*(y[j]-y[j-1])/delta
        #Also see what it is with the nominal change of the parameter
        #pdb.set_trace()

    
#EE    
    


NameError: name 'sign' is not defined

In [8]:
#Now we are going to try to parallelize it. For the package we are looking at (joblib) we need to put
#the code to compute the EE, etc. in a function
#def runEE(params,traj,EE,y,j):
def testEE(params,traj,j):
    #EE = []
    y = []
    sign = 0
    #First, change the input values
    x = params['thetam':].val*traj[j,:]
    #Second, run the model and put the output(s) in the y array
    #y[j] = x.sum()
    y = x.sum()
    #Then, see what changed and in which direction and calculate the elemental effect. 
    #Could pull from loop, but this isn't really going to impact performance on long model runs
    if j == 0: #No change in first run
        i = None
    else:#i is what input was changed
        #i is what input was changed          
        sign = np.sign((traj[j,:] - traj[j-1,:]))
        i = x.index[np.where(sign !=0)[0][0]]
        sign = sign.sum()
        #Calculate the elemental effect - using normalized change in parameter will always be delta (d)
        #pdb.set_trace()
        #EE = sign.sum()*(y[j]-y[j-1])/delta
    return y,sign,i

In [330]:
from joblib import Parallel, delayed
#Parallel processing!
traj = Bstar[17][0]
outs = pd.DataFrame(index=range(0,k),columns=['y','sign','param'])
#pdb.set_trace()
#Set up output dataframe to store the elemental effects
EE = pd.DataFrame(index = params['thetam':].index)
EE.loc[:,'EE'] = 0
delta = d
#outs.loc[j,['y','sign','param']] = Parallel(n_jobs=2)(delayed(runEE)(params,traj,j)for j in range(len(traj[:,0])))
res = Parallel(n_jobs=2)(delayed(testEE)(params,traj,j)for j in range(len(traj[:,0])))
for jind,j in enumerate(res):
    #pdb.set_trace()
    outs.loc[jind,['y','sign','param']] = j

In [303]:
#Regular old loop
outs = pd.DataFrame(index=range(0,k),columns=['y','sign','param'])
#pdb.set_trace()
#outs.loc[:,['y','sign','param']] = 0
#y = np.zeros((k+1,1))
#sign = np.zeros((k+1,1))
#param = np.empty((k+1,1),dtype=str)
#param = []
#outs = pd.DataFrame(y)
#EE = pd.DataFrame(index = params['thetam':].index)
#EE.loc[:,'EE'] = 0

for j in range(len(traj[:,0])):
    #pdb.set_trace()
    outs.loc[j,['y','sign','param']] = runEE(params,traj,j)


In [15]:
#Now we are going to make it so it actually runs the code. 
from BCBlues import BCBlues
from HelperFuncs import df_sliced_index
#def runEE(params,traj,EE,y,j):
#objparams = []
def BC_EE(params,traj,j,modparams):
    #pdb.set_trace()
    #EE = []
    y = []
    sign = 0
    #First, change the input values
    paramtest = params.copy(deep=True)
    x = paramtest['thetam':].val*traj[j,:]
    paramtest.loc['thetam':,'val'] = np.array(x)
    #Next, update the chemical parameters. Use Kow adjustment for KocW as well
    #pdb.set_trace()
    modparams[1].loc[:,'LogKoW'] = modparams[1].LogKocW*paramtest.val.Koc
    modparams[1].loc[:,'LogKocW'] = modparams[1].LogKocW*paramtest.val.Koc
    modparams[1].loc[:,'WatHL'] = modparams[1].LogKocW*paramtest.val.WatHL
    modparams[1].loc[:,'SoilHL'] = modparams[1].LogKocW*paramtest.val.SoilHL
    modparams[1].loc[:,'AirOHRateConst'] = modparams[1].LogKocW*params.val.AirOHRateConst
    
    #Initialize the model with the new parameters
    #modparams[0] is locsumm, 1 is chemsumm, 2 is timeseries, 3 is pp, 4 is numc
    #pdb.set_trace()
    kortright_BC = BCBlues(modparams[0],modparams[1],paramtest,modparams[2],modparams[4])
    #Run the model
    #pdb.set_trace()
    flow_time = kortright_BC.flow_time(modparams[0],paramtest,['water', 'subsoil'],modparams[2])
    mask = timeseries.time>=0
    minslice = np.min(np.where(mask))
    maxslice = np.max(np.where(mask))#minslice + 5 #
    flow_time = df_sliced_index(flow_time.loc[(slice(minslice,maxslice),slice(None)),:])
    input_calcs = kortright_BC.input_calc(modparams[0],modparams[1],paramtest,modparams[3],modparams[4],flow_time)
    model_outs = kortright_BC.run_it(modparams[0],modparams[1],paramtest,modparams[3],modparams[4],
                                     modparams[2],input_calcs)
    #Set what the tracked output variable(s) are
    #Lets set to proportion of mass flux out the end.
    
    numx = model_outs.index.levels[2][-1]
    N_effluent = np.array(model_outs.a1_t[slice(None),slice(None),numx-1]*model_outs.Z1[slice(None),slice(None),numx-1]\
                                            *model_outs.Qout[slice(None),slice(None),numx-1])
    #pdb.set_trace()
    dt = model_outs.time.groupby(level=1).mean()
    y = (N_effluent*dt).sum()/model_outs.Min.groupby(level=0).sum()
    #Then, see what changed and in which direction so that we can calculate elemental effects
    #Could pull from loop, but this isn't really going to impact performance on long model runs
    if j == 0: #First run
        #pdb.set_trace()
        i = None
        #flow_time = 
    else:#i is what input was changed
        #i is what input was changed          
        sign = np.sign((traj[j,:] - traj[j-1,:]))
        i = x.index[np.where(sign !=0)[0][0]]
        sign = sign.sum()
        #Calculate the elemental effect - using normalized change in parameter will always be delta (d)
        #pdb.set_trace()
        #EE = sign.sum()*(y[j]-y[j-1])/delta
    return y,sign,i

In [9]:
import BCBlues
import importlib
importlib.reload(BCBlues)

<module 'BCBlues' from 'D:\\Users\\Tim Rodgers\\Documents\\GitHub\\BioretentionBlues\\BCBlues.py'>

In [16]:
from BCBlues import BCBlues
from joblib import Parallel, delayed
params = pd.read_excel('params_BC_sensitivity.xlsx',index_col = 0)
locsumm = pd.read_excel('Kortright_BC.xlsx',index_col = 0)
chemsumm = pd.read_excel('sensitivity_CHEMSUMM.xlsx',index_col = 0)
timeseries = pd.read_excel('timeseries_tracertest_Kortright_sensitivity.xlsx')
pp = None
numc = ['water', 'subsoil','rootbody', 'rootxylem', 'rootcyl','shoots', 'air', 'pond']
modparams = [locsumm,chemsumm,timeseries,pp,numc]
#kortright_BC = BCBlues(modparams[0],modparams[1],params,modparams[2],modparams[4])
#Initialize the flow_time out

pdb.set_trace()
#Choose the trajectory

traj = Bstar[8][0]
j = 0
#Now, run it
#BC_EE(params,traj,j,modparams)
pdb.set_trace()
res = Parallel(n_jobs=2)(delayed(BC_EE)(params,traj,j,modparams)for j in range(len(traj[:,0])))
#res = Parallel(n_jobs=2)(delayed(testEE)(params,traj,j)for j in range(len(traj[:,0])))    



--Return--
> <ipython-input-16-a72efa8f4fc8>(13)<module>()->None
-> pdb.set_trace()
(Pdb) 
(Pdb) c
--Return--
> <ipython-input-16-a72efa8f4fc8>(20)<module>()->None
-> pdb.set_trace()
(Pdb) c


In [17]:
res

[(EHDPP    0.000079
  Name: Min, dtype: float64,
  0,
  None),
 (EHDPP    0.000079
  Name: Min, dtype: float64,
  -1.0,
  'OHConc'),
 (EHDPP    0.000079
  Name: Min, dtype: float64,
  1.0,
  'AirOHRateConst'),
 (EHDPP    0.000079
  Name: Min, dtype: float64,
  1.0,
  'Ifw'),
 (EHDPP    0.000079
  Name: Min, dtype: float64,
  -1.0,
  'lamb'),
 (EHDPP    0.000079
  Name: Min, dtype: float64,
  -1.0,
  'Up'),
 (EHDPP    0.000079
  Name: Min, dtype: float64,
  1.0,
  'LAI'),
 (EHDPP    0.000081
  Name: Min, dtype: float64,
  -1.0,
  'Kn'),
 (EHDPP    0.000081
  Name: Min, dtype: float64,
  -1.0,
  'VFrootcyl'),
 (EHDPP    1.075326e-07
  Name: Min, dtype: float64,
  1.0,
  'thetam'),
 (EHDPP    1.075326e-07
  Name: Min, dtype: float64,
  -1.0,
  'kwe'),
 (EHDPP    1.075326e-07
  Name: Min, dtype: float64,
  -1.0,
  'VFrootxylem'),
 (EHDPP    0.000004
  Name: Min, dtype: float64,
  1.0,
  'alpha'),
 (EHDPP    0.000004
  Name: Min, dtype: float64,
  1.0,
  'Kn2'),
 (EHDPP    0.000004
  Name: 

In [19]:
B78_res = np.load('B78_res.npy',allow_pickle=True)

ValueError: can only convert an array of size 1 to a Python scalar

In [18]:
B15_res

RecursionError: maximum recursion depth exceeded while calling a Python object